# Check Dataset

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from PIL import Image
import os
import os
from os import listdir
from os.path import isfile, join

In [9]:
os.environ['IMAGES_FOLDER']
#print(os.environ)

'/Users/nataliamenacho/code/NMenacho/to_infinity_and_beyond/To-infinity-and-beyond/data/images_cropped_sample'

In [10]:
image_dir = os.environ['IMAGES_FOLDER']

#image_files = [f for f in listdir(image_dir) if isfile(join(image_dir, f))]

image_files = []
for f in listdir(image_dir):
    if isfile(os.path.join(image_dir, f)):
        if f.endswith(".jpg"):
            image_files.append(f)
        else:
            print("not jpg ignored")

X_list = []
y_list = []

for image_file in image_files:
    image_info = image_file.replace('.jpg', '').split("_")
    cat = image_info[3]
    image_data = np.array(Image.open(os.path.join(image_dir,image_file)))
    #print(image_info)
    #print(image_data.shape)
    X_list.append(image_data)
    y_list.append(cat)

X_data = np.array(X_list)
print(X_data.shape)
y_data = np.array(y_list)
print(y_data.shape)

not jpg ignored
not jpg ignored
(1370, 128, 128, 3)
(1370,)


In [4]:
#plt.imshow(X_data[98]) / use this tomorrow

In [40]:
#y.value_counts()/len(y)

1    0.691241
0    0.308759
Name: count, dtype: float64

In [11]:
import pandas as pd
y_data_cat = pd.Series(y_data).map({'STAR':0, 'GALAXY':1})
y_data_cat


0       0
1       1
2       1
3       0
4       1
       ..
1365    0
1366    1
1367    0
1368    0
1369    0
Length: 1370, dtype: int64

In [12]:
import pandas as pd
from tensorflow.keras.utils import to_categorical

#df = pd.read_csv('/code/NMenacho/to_infinity_and_beyond/To-infinity-and-beyond/data')
X = X_data
y = y_data_cat

y_cat = to_categorical(y, num_classes=2)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X,y_cat,
                                                    test_size=0.2,
                                                    random_state=42)


In [14]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1096, 128, 128, 3), (274, 128, 128, 3), (1096, 2), (274, 2))

# The Convolutional Neural Network

### A. Architecture and compilation of a CNN

In [21]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(2, activation='softmax')
])

In [22]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │       576,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │            22 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 595,424 (2.27 MB)

 Trainable params: 595,424 (2.27 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
#Compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


### B. Training the CNN

In [24]:
#Train the model
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5)

history = model.fit(X_train, y_train,
          batch_size=32,
          epochs=100,
          validation_split=0.2,
          callbacks=[es])


Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 100ms/step - accuracy: 0.6908 - loss: 6.3917 - val_accuracy: 0.9318 - val_loss: 0.3707
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 97ms/step - accuracy: 0.9212 - loss: 0.3416 - val_accuracy: 0.9273 - val_loss: 0.2465
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 96ms/step - accuracy: 0.9365 - loss: 0.2366 - val_accuracy: 0.9227 - val_loss: 0.2668
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 90ms/step - accuracy: 0.9409 - loss: 0.2416 - val_accuracy: 0.9227 - val_loss: 0.2175
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.9396 - loss: 0.1748 - val_accuracy: 0.9273 - val_loss: 0.2192
Epoch 6/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.9420 - loss: 0.1781 - val_accuracy: 0.9273 - val_loss: 0.2271
Epoch 7/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step - accuracy: 0.9626 - loss: 0.1286 - val_accuracy: 0.9136 - val_loss: 0.2640
Epoch 8/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.9611 - loss: 0.1144 - val_accuracy: 0

### C. Evaluate model's performance

In [25]:
# Evaluate the model
model.evaluate(X_test, y_test)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9179 - loss: 0.2393


[0.1968117356300354, 0.9416058659553528]